<a href="https://colab.research.google.com/github/nestorap/food-not-food/blob/main/Preprocesamiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/nestorap/food-not-food.git

In [130]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import csv
import ast
from nltk.corpus import wordnet as wn
import nltk
import random

In [ ]:
# Descargamos palabrasgo obtener las que son de comida

nltk.download('popular')

In [4]:
# Aquí leemos los nombres de las clases de las imágenes

with open("food-not-food/imges_list.txt", "r") as f:
  images_classes = ast.literal_eval(f.read())

In [5]:
# Recorremos los items del diccionario

for k, v in images_classes.items():
  if "banana" in v:
    print(k)



954


## Nombre de comidas de nltk

In [6]:
# Obtenemos las palabras de comida del dataset de nltk
# Con esta lista podemos filtrar las imágenes para sacar aquellas fotos que sean de comida y que no
food = wn.synset("food.n.02")
food_list = list(set([w for s in food.closure(lambda s: s.hyponyms()) for w in s.lemma_names()])) #hyponyms parece ser que significa sinónimos


In [7]:
# Check para banana
"banana" in food_list

True

## Filtramos los items para que coincidan a la hora de descargar

In [8]:
# limpiamos los string para eliminar algunos signos de puntuación

lista_food = [food_item.lower().split("_") for food_item in food_list]

In [9]:
# Unimos las palabras en una misma lista. La celda de antes obtenemos una lista de listas de palabras

food_lista = [food for food_sublist in lista_food for food in food_sublist]

In [10]:
 # checkeamos

 for k,v in images_classes.items():
   if v.lower() in food_lista:
     print(k, v)

8 hen
85 quail
86 partridge
99 goose
113 snail
331 hare
339 sorrel
390 eel
599 honeycomb
723 pinwheel
923 plate
932 pretzel
937 broccoli
938 cauliflower
946 cardoon
947 mushroom
949 strawberry
950 orange
951 lemon
952 fig
954 banana
957 pomegranate
968 cup
987 corn
988 acorn


In [11]:
imagenet_food_clases = dict()

for k,v in images_classes.items():
  imagenet_class_set = set([space_word.strip(" ") for space_word in v.lower().split(",")]) 
  if imagenet_class_set.intersection(food_lista):
    imagenet_food_clases[k] = v

In [12]:
# Una vez que tenemos las clases vamos a ver cuales no deberian estar aqui. Vamos a filtrar manualmente y poner los indices que no deberian ser comida

no_food = [457, 494, 567, 626, 723, 738, 760, 923, 972, 976]

In [13]:
# Del diccionario que hemos obtenido, eliminamos aquellas llaves que no sean comida

imagenet_manual_filter_clases = dict()

for k,v in imagenet_food_clases.items():
  if k not in no_food: # no queremos las llaves que no sean comida
    imagenet_manual_filter_clases[k] = v

In [14]:
# Vamos a sacar una lista de las llaves del diccionario de comida para mas tarde filtrar y sacar las clases que no son comida

llaves_comida = [i for i in imagenet_manual_filter_clases.keys()]

## Sacaremos lista de comida y no-comida de imagenet

In [15]:
imagenet_non_food_clases = dict()

for k,v in images_classes.items():
  if k not in llaves_comida:
    imagenet_non_food_clases[k] = v

## Siguientes fase del proyecto
1- Hemos filtrado las comidas y las no-comidas del dataset con los nombres y sus índices. \
2- Ahora parece ser que tenemos que seguir otra regla para que Imagenet entienda las clases que queremos. \
3- Hay que leer un csv de github que aparece en el tutorial \
4- Volver 10 min atras en el video para recordar lo que estaba haciendo

----

## Imagenet keys

El script para descargar las imágenes utiliza las llaves de las clases de Imagenet. Por ejemplo --> n09858165 n01539573 n03405111.  
Vamos a mapear la lista para sacar esas llaves o ids.

Imagenet data downloader script - https://github.com/mf1024/ImageNet-Datasets-Downloader

In [16]:
# Aquí tenemos las imagenes de Imaganet con sus clases y llaves

df = pd.read_csv("https://raw.githubusercontent.com/mf1024/ImageNet-Datasets-Downloader/master/classes_in_imagenet.csv")
df["class_name"] = df.loc[:,"class_name"].str.lower()

In [17]:
 # Borramos los nans para que no nos fastidien al filtrar

df.dropna(inplace=True)

In [18]:
# Filtramos el dataframe con las clases que tenemos en la lista de food_list

df_no_food = df.loc[~df["class_name"].isin(food_lista),:]
df_food = df.loc[df["class_name"].isin(food_lista),:]

In [19]:
# Eliminamos las clases que no son comida del df_food
# Lo haremos de manera manual. Mejor hacerlo de manera automática siempre que se pueda

no_food_list = [
                "ball",
                "puppy",
                "dog",
                "game",
                "bar",
                "blade",
                "garden",
                "hand",
                "head",
                "jacket",
                "junk",
                "key",
                "leg",
                "oven",
                "pin",
                "pinwheel",
                "plate",
                "pot",
                "rack",
                "refrigerator",
                "saddle",
                "shank",
                "spring",
                "steamer",
                "stick",
                "temple",
                "turck",
                "turban",
                "ring",
                "cup",
                "rock",
                "shell",
                "pilot",
                "runner",
                "smith",
                "ash",
                "sand"
]



In [20]:
# En la lista anterior tenemos nombres que no son comida en la lista que sí lo son. Esos nombres los vamos a filtrar para quitarlo del df de comida

df_food = df_food.loc[(~df["class_name"].isin(no_food_list)),:]
df_food.shape[0]

861

In [29]:
# Hacemos lista de food clases

# Lista de los ids de las clases
imagenet_food_class_ids = df_food.loc[:,"synid"].tolist()
# Lista de los nombres de las clases
imagenet_food_class_names = df_food.loc[:,"class_name"].tolist()
# Unimos ids y nombres en un diccionario
imagenet_food_class_name_and_ids_dict = dict(zip(imagenet_food_class_ids, imagenet_food_class_names))

In [30]:
# Hacemos lista de no_food clases

# Lista de los ids de las clases
imagenet_no_food_class_ids = df_no_food.loc[:,"synid"].tolist()
# Lista de los nombres de las clases
imagenet_no_food_class_names = df_no_food.loc[:,"class_name"].tolist()
# Unimos ids y nombres en un diccionario
imagenet_no_food_class_name_and_ids_dict = dict(zip(imagenet_no_food_class_ids, imagenet_no_food_class_names))

In [75]:
# Lista de los ids de las clases de food
food_class_id_list = list(imagenet_food_class_name_and_ids_dict.keys())

# Lista de los ids de las clases de no_food
no_food_class_id_list = list(imagenet_no_food_class_name_and_ids_dict.keys())


In [77]:
# Hacemos funcion para convertir la lista en string para pasarlo al script

def convert_list(list):
  empty_string = ""
  for item in list:
    empty_string += " "+item
  empty_string = empty_string[1:]
  return empty_string

# Obtenemos los ids para pasarle al script del downloader
food_class_id_string = convert_list(food_class_id_list)
no_food_class_id_string = convert_list(no_food_class_id_list)

# Descargamos los datos con comandos en bash
En esta parte vamos a descargar las imágenes del repo de github
Copiamos dos carpetas con imagenes en el directorio "model_test_images" para hacer una red neuronal pequeña. \
Copiamos 2 directorios del dataset descargado de manera random con el siguiente comando en la consola --> cp -r imagenet_images/imagenet_images/'African violet' model_test_images

In [ ]:
! git clone https://github.com/mf1024/ImageNet-Datasets-Downloader.git

In [37]:
from IPython.display import JSON
from google.colab import output
from subprocess import getoutput
import os

def shell(command):
  if command.startswith('cd'):
    path = command.strip().split(maxsplit=1)[1]
    os.chdir(path)
    return JSON([''])
  return JSON([getoutput(command)])
output.register_callback('shell', shell)

In [119]:
#@title Colab Shell
%%html
<div id=term_demo></div>
<script src="https://code.jquery.com/jquery-latest.js"></script>
<script src="https://cdn.jsdelivr.net/npm/jquery.terminal/js/jquery.terminal.min.js"></script>
<link href="https://cdn.jsdelivr.net/npm/jquery.terminal/css/jquery.terminal.min.css" rel="stylesheet"/>
<script>
  $('#term_demo').terminal(async function(command) {
      if (command !== '') {
          try {
              let res = await google.colab.kernel.invokeFunction('shell', [command])
              let out = res.data['application/json'][0]
              this.echo(new String(out))
          } catch(e) {
              this.error(new String(e));
          }
      } else {
          this.echo('');
      }
  }, {
      greetings: 'Welcome to Colab Shell',
      name: 'colab_demo',
      height: 250,
      prompt: 'colab > '
  });

# Descarga de los datos en el jupyter
Aquí, con código en una celda de jupyter, también podemos descargar las imágenes sin ncesidad de abrir una consola

In [95]:
/content/food-not-food/data/imagenet_images/imagenet_images/Beaujolais

# Creamos los directorios de las imágenes
!mkdir data/

In [96]:
# Creamos los directorios para los datos de comida y de no comida
!mkdir data/food_images
!mkdir data/no_food_images

# Creamos un directorio para imágenes random
!mkdir food-not-food/data/imagenet_images

In [70]:
# Descargamos las imágenes de comida
!python ImageNet-Datasets-Downloader/downloader.py \-data_root food-not-food/data/food_images \-use_class_list=True \-class_list $food_class_id_string \-images_per_class 100


ERROR:root:Class [i not found in ImageNete


In [ ]:
# DEscargamos las imágenes de no_food
!python ImageNet-Datasets-Downloader/downloader.py \-data_root food-not-food/data/imagenet_images \-number_of_classes 1000 \-images_per_class 50

# Construcción del modelo

In [ ]:
!pip install tflite_model_maker

In [118]:
import tensorflow as tf
import tflite_model_maker

### Creamos el train y test de los datos

In [127]:
# Creamos el directorio de train
!mkdir model_test_images/train

# Creamos el directorio de test
!mkdir model_test_images/test

In [129]:
# Listamos las imágenes de los distintas muestras
beau_list = os.listdir("model_test_images/Beaujolais")
violet_list = os.listdir("model_test_images/African violet")

In [136]:
# Hacemos split

def train_test_list(target_dir):
  image_list = os.listdir(target_dir)

  random.seed(42)
  train_split = int(0.8 * len(image_list))
  # Sacamos una muestra random
  train_image_list = random.sample(image_list, train_split)
  test_image_list = list(set(image_list).difference(set(train_image_list)))

  return train_image_list, test_image_list

# Sacamos el train y el test
train_image_list, test_image_list = train_test_list("model_test_images/Beaujolais")

45 12


In [122]:
# Nos movemos al directorio donde tenemos las imágenes de test

os.walk("data/model_test_images")

<generator object walk at 0x7f155db75dd0>

In [135]:
os.getcwd()

'/content/food-not-food/data'